In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import expon
from scipy.stats import lognorm
from scipy.stats import powerlaw
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from itertools import combinations
%config InlineBackend.figure_format = 'retina'
%pylab inline
import zipfile
import networkx as nx#import building data 



Populating the interactive namespace from numpy and matplotlib


In [42]:
#import data from Building Assessors for 2017
df = pd.read_csv('/Users/arianna/Dropbox (MIT)/PHD/Spring_2018/11.S941_Big_Data_Visualization_Society/BDVS_Team Collaboration/data/raw_data/property-assessment-fy2017.csv')
df.head()


,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,...,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
0,0100001000_,NaN,0100001000_,104 A 104,PUTNAM,ST,NaN,02128_,105,R3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0100002000_,NaN,0100002000_,197,LEXINGTON,ST,NaN,02128_,105,R3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0100003000_,NaN,0100003000_,199,LEXINGTON,ST,NaN,02128_,105,R3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0100004000_,NaN,0100004000_,201,LEXINGTON,ST,NaN,02128_,105,R3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0100005000_,NaN,0100005000_,203,LEXINGTON,ST,NaN,02128_,104,R2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#list the column names
list(df)

['PID',
 'CM_ID',
 'GIS_ID',
 'ST_NUM',
 'ST_NAME',
 'ST_NAME_SUF',
 'UNIT_NUM',
 'ZIPCODE',
 'PTYPE',
 'LU',
 'OWN_OCC',
 'OWNER',
 'MAIL_ADDRESSEE',
 'MAIL_ADDRESS',
 'MAIL CS',
 'MAIL_ZIPCODE',
 'AV_LAND',
 'AV_BLDG',
 'AV_TOTAL',
 'GROSS_TAX',
 'LAND_SF',
 'YR_BUILT',
 'YR_REMOD',
 'GROSS_AREA',
 'LIVING_AREA',
 'NUM_FLOORS',
 'STRUCTURE_CLASS',
 'R_BLDG_STYL',
 'R_ROOF_TYP',
 'R_EXT_FIN',
 'R_TOTAL_RMS',
 'R_BDRMS',
 'R_FULL_BTH',
 'R_HALF_BTH',
 'R_BTH_STYLE',
 'R_BTH_STYLE2',
 'R_BTH_STYLE3',
 'R_KITCH',
 'R_KITCH_STYLE',
 'R_KITCH_STYLE2',
 'R_KITCH_STYLE3',
 'R_HEAT_TYP',
 'R_AC',
 'R_FPLACE',
 'R_EXT_CND',
 'R_OVRALL_CND',
 'R_INT_CND',
 'R_INT_FIN',
 'R_VIEW',
 'S_NUM_BLDG',
 'S_BLDG_STYL',
 'S_UNIT_RES',
 'S_UNIT_COM',
 'S_UNIT_RC',
 'S_EXT_FIN',
 'S_EXT_CND',
 'U_BASE_FLOOR',
 'U_NUM_PARK',
 'U_CORNER',
 'U_ORIENT',
 'U_TOT_RMS',
 'U_BDRMS',
 'U_FULL_BTH',
 'U_HALF_BTH',
 'U_BTH_STYLE',
 'U_BTH_STYLE2',
 'U_BTH_STYLE3',
 'U_KITCH_TYPE',
 'U_KITCH_STYLE',
 'U_HEAT_TYP',
 'U

In [44]:
df['GROSS_AREA'].max()

1976650.0

In [45]:
df['GROSS_AREA'].min()

0.0

In [46]:
df['GROSS_AREA'].mean()

4519.714134843866

In [47]:
#Keep only variables of interest 
df_main_variables= df[['PID','GIS_ID','ZIPCODE', 'PTYPE', 'AV_LAND', 'LAND_SF', 'YR_BUILT', 'GROSS_AREA', 'NUM_FLOORS','STRUCTURE_CLASS', 'R_EXT_FIN','R_BLDG_STYL', 'R_HEAT_TYP','R_OVRALL_CND', 'R_INT_CND', 'S_BLDG_STYL', 'S_EXT_FIN', 'S_EXT_CND', 'U_HEAT_TYP', 'U_AC',  
 'R_AC',
 'R_ROOF_TYP' ]]

In [48]:
#leave only observations that are larger than 50,000 (for commercial properties)
qualified = df_main_variables['GROSS_AREA'] > 50000

In [49]:
df_clean = df_main_variables[qualified]

In [53]:
df_clean.head()

,PID,GIS_ID,ZIPCODE,PTYPE,AV_LAND,LAND_SF,YR_BUILT,GROSS_AREA,NUM_FLOORS,STRUCTURE_CLASS,...,R_HEAT_TYP,R_OVRALL_CND,R_INT_CND,S_BLDG_STYL,S_EXT_FIN,S_EXT_CND,U_HEAT_TYP,U_AC,R_AC,R_ROOF_TYP
16,0401753000_,0401753000_,02115_,977,12885200,25274.0,1899.0,99720.0,5.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0401755000_,0401755000_,02215_,977,9070500,13318.0,1899.0,75457.0,6.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,0401777000_,0401777000_,02115_,31,3315400,12000.0,1900.0,66523.0,5.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,0401796000_,0401796000_,02115_,905,116294800,502965.0,1900.0,1232512.0,2.5,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,0401797000_,0401797000_,02115_,907,2219621,6839.0,1910.0,156761.0,5.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
#Export file to clean data folder. 
df_clean.to_csv('/Users/arianna/Dropbox (MIT)/PHD/Spring_2018/11.S941_Big_Data_Visualization_Society/BDVS_Team Collaboration/data/clean_data/Building_Skeleton_Assessors_2017.csv') 